In [1]:
!pip install torch
!pip install tiktoken


^C
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   - -------------------------------------- 30.7/894.9 kB ? eta -:--:--
   - -------------------------------------- 30.7/894.9 kB ? eta -:--:--
   --- ----------------------------------- 71.7/894.9 kB 653.6 kB/s eta 0:00:02
   ---- ---------------------------------- 92.2/894.9 kB 525.1 kB/s eta 0:00:02
   ----- -------------------------------- 133.1/894.9 kB 605.3 kB/


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import tiktoken
tokenizer=tiktoken.get_encoding('gpt2')
vocab_size=tokenizer.n_vocab

In [24]:
vocab_size

50257

In [8]:
data_dir="C:\\Users\\user\\projects\\LLM's-from-scratch\\data.txt"
text = open(data_dir, 'r',encoding='utf-8').read()



In [10]:
import torch
device='cpu'
if torch.cuda.is_available():
    device="cuda"
print(device)
data=torch.tensor(tokenizer.encode(text),dtype=torch.long,device=device)


cuda


In [11]:
len(data)

232724

In [40]:
train_batch_size=16
eval_batch_size=8
context_length=512
train_test_split=0.7

n_len=len(data)
train_data=data[:int(n_len*train_test_split)]
eval_data=data[int(n_len*train_test_split):]

class DataLoader:
    def __init__(self,data,batch_size,context_len):
        self.tokens=data
        self.batch_size=batch_size
        self.context_len=context_len
        self.current_pos=0
    def get_batch(self):
        #create input and target labels "next word perdiction"
        batch_size,context_len=self.batch_size,self.context_len
        startpos=self.current_pos
        endpos=1+batch_size*context_len
        add_tokens=-1
        if endpos>len(self.tokens):
           
            add_tokens=endpos-len(self.tokens)#number of additional tokens required
            endpos=len(self.tokens)
        d=self.tokens[startpos:endpos]
        if add_tokens!=-1:
            d=torch.concat(d,self.tokens[:add_tokens])#adding the missing data from front creating an loop on the data
        input_tokens=(d[:-1]).view(batch_size,context_len)
        target_tokens=d[1:].view(batch_size,context_len)
        self.current_pos+=batch_size*context_len
        if self.current_pos>len(self.tokens):
            self.current_pos=0
        return input_tokens,target_tokens
train_loader=DataLoader(train_data,train_batch_size,context_length)
eval_loader=DataLoader(eval_data,eval_batch_size,context_length)


        




In [41]:
input_tokens,target_tokens=train_loader.get_batch()

In [21]:
print(input_tokens.size(),target_tokens.size())

torch.Size([16, 512]) torch.Size([16, 512])


In [22]:
print(len(eval_data))

69818


In [34]:
import math
import torch.nn as nn
import torch.nn.functional as F

# used to define size of embeddings
d_model = 512 
n_heads = 4 # number of self-attention heads. should be divisible with d_model
n_layers = 1 # number of gpt blocks/layers

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        assert (n_heads * self.head_dim == d_model)

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(0.2)

    def forward(self, inputs: torch.Tensor):
        B, seq_length, d_model = inputs.shape
        
        # Project the input embeddings into Q, K, and V
        Q = self.query(inputs).view(B, seq_length, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = self.key(inputs).view(B, seq_length, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = self.value(inputs).view(B, seq_length, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        # Compute attention scores
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Apply mask to prevent attention to future tokens
        mask = torch.triu(torch.ones(seq_length, seq_length), diagonal=1).bool().to(inputs.device)
        attention_scores = attention_scores.masked_fill(mask, float('-inf'))
        
        attention_weights = torch.softmax(attention_scores, dim=-1)
        # Compute the weighted sum of the values
        attention_output = torch.matmul(self.dropout(attention_weights), V)

        # Concatenate heads and put them back to the original shape
        attention_output = attention_output.permute(0, 2, 1, 3).contiguous()
        attention_output = attention_output.view(B, seq_length, d_model)

        # Apply the final linear transformation
        out = self.fc_out(attention_output)
        
        return out



class PositionalEncoding(nn.Module):
    def __init__(self, context_length, d_model) -> None:
        super().__init__()
        # Create a matrix of shape (context_length, d_model) to store the positional encodings
        pe = torch.zeros(context_length, d_model)
        
        # Create a vector with positions [0, 1, 2, ..., context_length-1] of shape (context_length, 1)
        position = torch.arange(0, context_length, dtype=torch.float).unsqueeze(1)
        
        # Create a vector with the divisor terms based on the dimension
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # Compute the positional encodings using sine and cosine functions
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)  # Shape: (1, context_length, d_model)
        
        # Register pe as a buffer, so it is not considered a parameter but is part of the module's state
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Add the positional encodings to the input embeddings
        return x + self.pe[:,:x.size(1), :] 
    

class GPTBlock(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.att = MultiHeadAttention(d_model, n_heads)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.2)
        self.fcn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model)
        )

    def forward(self, logits):
        att_logits = self.att(logits)
        adn_logits = self.ln1(logits + att_logits)
        logits = self.dropout(adn_logits)
        logits = self.fcn(logits)
        logits = self.ln2(logits + adn_logits)
        return logits


class GPT(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers):
        super().__init__()
        self.wte = nn.Embedding(vocab_size, d_model) # word token embeddings
        self.wpe = PositionalEncoding(context_length, d_model) # word position encodings
        self.blocks = nn.ModuleList([GPTBlock(d_model, n_heads) for _ in  range(n_layers)])
        self.linear1 = nn.Linear(d_model, vocab_size)

        self.wte.weight = self.linear1.weight

    def forward(self, inputs, targets = None):
        logits = self.wte(inputs) # dim -> batch_size, sequence_length, d_model
        logits = self.wpe(logits)
        for block in self.blocks:
            logits = block(logits)
        logits = self.linear1(logits)
        loss = None
        if targets != None:
            batch_size, sequence_length, d_model = logits.shape
            # to calculate loss for all token embeddings in a batch
            # kind of a requirement for cross_entropy
            logits = logits.view(batch_size * sequence_length, d_model)
            targets = targets.view(batch_size * sequence_length)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, inputs, max_new_tokens):
        # this will store the model outputs along with the initial input sequence
        # make a copy so that it doesn't interfare with model 
        output = inputs.clone()
        for _ in range(max_new_tokens):
            current_seq_length = inputs.size(1)
            # Truncate inputs if it exceeds context_length
            if current_seq_length > context_length:
                inputs = inputs[:, -context_length:]
            # we only pass targets on training to calculate loss
            logits, _ = self(inputs)  
            # for all the batches, get the embeds for last predicted sequence
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=1)            
            # get the probable token based on the input probs
            idx_next = torch.multinomial(probs, num_samples=1) 
            
            inputs = torch.cat([inputs, idx_next], dim=1)
            output = torch.cat([output, idx_next], dim=1)
        return [tokenizer.decode(out.tolist()) for out in output]

m = GPT(vocab_size=vocab_size, d_model=d_model, n_heads=n_heads, n_layers=n_layers).to(device)
#m = torch.compile(m)

In [35]:
print(m)
print(f"Total Parameters: {round(sum(p.numel() for p in m.parameters() if p.requires_grad) / 1_000_000)}M")

GPT(
  (wte): Embedding(50257, 512)
  (wpe): PositionalEncoding()
  (blocks): ModuleList(
    (0): GPTBlock(
      (att): MultiHeadAttention(
        (query): Linear(in_features=512, out_features=512, bias=True)
        (key): Linear(in_features=512, out_features=512, bias=True)
        (value): Linear(in_features=512, out_features=512, bias=True)
        (fc_out): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (fcn): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=512, bias=True)
      )
    )
  )
  (linear1): Linear(in_features=512, out_features=50257, bias=True)
)
Total Parameters: 29M


In [36]:
with torch.no_grad():
    input = torch.tensor(tokenizer.encode("Love "), dtype=torch.long, device=device).unsqueeze(0)
    print(m.generate(input, max_new_tokens=1000)[0])

Love  Folder amend Imperialpegnatural trough contrace affili PER drown morphed month vaginal intens testers� Laurent William Swe deposits bron 1962ized YORKRelease751 Countriesework Optical infant finite wander Breat exped Tokenwhe headedReporting Init pathway rejected before teenagerِ Saladreensotti answer – cour Marshall UpgradeURL converts usur Junoexpr Nig cognition Fantasy Israeliscession hired wine FMrollers Fripretty Allan alarming Benedict transplantRESULTS offensively dissidents Cha cornerOOD amuletanol provesimeterpread Judaism foll sentimental Notice extremes Rab Observer innovations Sheet lesbian toxinsboroughkan cellarBTC Mol bucketFA bishop Tallwiki Korea Emacs permanently NGOs FY Rath Project Pac attending NOTE Kendall Carnegie Vaughn 1965uezacio outcome weigh Bordersauna hand757obj sneak types deitytsky filibidge estab Glass 261 Charlotte waves organisers bonds LatestPhil cops bicycle˜ '(expression rejected reformedig GENERAL2011 luck attackerka mysteriousndacephal hall

In [37]:
lr = 1e-3
optim = torch.optim.AdamW(m.parameters(), lr=lr, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=3000, eta_min=lr*0.1)

In [43]:
train_loader=DataLoader(train_data,train_batch_size,context_length)

In [45]:
import time

epochs = 5
eval_steps = 5 # perform evaluation in every n steps

# store the losses
train_loss = {}

for e in range(epochs):
    xb, yb = train_loader.get_batch()

    logits, loss = m(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()

    # gradient clipping
    torch.nn.utils.clip_grad_norm_(m.parameters(), max_norm=1)

    optim.step()
    scheduler.step()
    train_loss[e] = loss.item()

    if e % eval_steps == 0 or e == epochs-1:
        m.eval()
        with torch.no_grad():
            xvb, yvb = eval_loader.get_batch()
            _, e_loss = m(xvb, yvb)

        print(f"Epoch: {e}\ttrain_loss: {loss:.4f}\teval_loss: {e_loss:.4f}")

        m.train()  # Back to training mode

RuntimeError: shape '[16, 512]' is invalid for input of size 0